In [1]:
import os

from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

import sys
sys.path.append('/home/qinxie/2018/2018 second semester/2018LRP/LRP explain time series dataset/LRP explain time series dataset/Prepare v1/models')
from models_2_2 import MNIST_DNN
from utils import pixel_range


/home/qinxie/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# pwd

In [3]:
%matplotlib inline

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
images = mnist.train.images
labels = mnist.train.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
logdir = 'tf_logs/STD_V5'
ckptdir = logdir + '/model'

if not os.path.exists(logdir):
    os.makedirs(logdir)

<font color=blue>__Building Graph__</font>

In this step, a regular DNN classifier is initialized and necessary nodes for model training is added onto the graph. In this particular tutorial, I used a fully connected neural network with ReLU activations and without bias in order to satisfy the $f(tx) = tf(x)$ property mentioned in the introduction.

In [5]:
with tf.name_scope('Classifier'):
    
    #initial neural network
    DNN = MNIST_DNN('DNN')
    
    #setup training process
    X = tf.placeholder(tf.float32, [None, 784], name='X')
    Y = tf.placeholder(tf.float32, [None, 10], name='Y')

    activations,logits = DNN(X)
    
    tf.add_to_collection('STD', X)
#     tf.add_to_collection('STD', logits)
    for activation in activations:
        tf.add_to_collection('STD', activation)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    
    optimizer = tf.train.AdamOptimizer().minimize(cost, var_list=DNN.vars)
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
cost_summary = tf.summary.scalar('Cost', cost)
accracy_summary = tf.summary.scalar('Accuracy', accuracy)
summary = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



<font color=blue>__Training Network__</font>

This is the step where the DNN is trained to classify the 10 digits of the MNIST images. Summaries are written into the logdir and you can visualize the statistics using tensorboard by typing this command: tensorboard --lodir=/tf_logs

In [6]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [7]:
saver = tf.train.Saver()

In [8]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

#hyper parameters
training_epochs = 15
batch_size = 100

for epoch in range(training_epochs):
    total_batch = int(mnist.train.num_examples / batch_size)
    avg_cost = 0
    avg_acc = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c, a, summary_str = sess.run([optimizer, cost, accuracy, summary], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c / total_batch
        avg_acc += a / total_batch
        
        file_writer.add_summary(summary_str, epoch * total_batch + i)
        
    print('epoch:', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost), 'accuracy = ', '{:.9f}'.format(avg_acc))
    
    saver.save(sess, ckptdir)
    
print('accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

sess.close()

('epoch:', '0001', 'cost = ', '0.222180229', 'accuracy = ', '0.932200004')
('epoch:', '0002', 'cost = ', '0.096898515', 'accuracy = ', '0.971072736')
('epoch:', '0003', 'cost = ', '0.065677645', 'accuracy = ', '0.980054556')
('epoch:', '0004', 'cost = ', '0.051179192', 'accuracy = ', '0.983945466')
('epoch:', '0005', 'cost = ', '0.040895103', 'accuracy = ', '0.987672735')
('epoch:', '0006', 'cost = ', '0.034696673', 'accuracy = ', '0.989454554')
('epoch:', '0007', 'cost = ', '0.030927487', 'accuracy = ', '0.990872735')
('epoch:', '0008', 'cost = ', '0.028013923', 'accuracy = ', '0.991581825')
('epoch:', '0009', 'cost = ', '0.024057186', 'accuracy = ', '0.992872734')
('epoch:', '0010', 'cost = ', '0.022129803', 'accuracy = ', '0.993490915')
('epoch:', '0011', 'cost = ', '0.019094508', 'accuracy = ', '0.994109097')
('epoch:', '0012', 'cost = ', '0.017918524', 'accuracy = ', '0.994600005')
('epoch:', '0013', 'cost = ', '0.016289875', 'accuracy = ', '0.995418186')
('epoch:', '0014', 'cost 

<font color=blue>__Restoring Subgraph__</font>

In [9]:
tf.reset_default_graph()

sess = tf.InteractiveSession()

In [10]:
new_saver = tf.train.import_meta_graph(ckptdir+'.meta')

In [11]:
new_saver.restore(sess, tf.train.latest_checkpoint(logdir))

INFO:tensorflow:Restoring parameters from tf_logs/STD_V5/model


In [12]:
weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='DNN')
activations = tf.get_collection('STD')
X = activations[0]

In [13]:
print(X)
print
for i in range(len(weights)):
    print(weights[i])
print
for i in range(len(activations)):
    print('activations',activations[i])

Tensor("Classifier/X:0", shape=(?, 784), dtype=float32)

<tf.Variable 'DNN/dense/kernel:0' shape=(784, 512) dtype=float32_ref>
<tf.Variable 'DNN/dense_1/kernel:0' shape=(512, 512) dtype=float32_ref>
<tf.Variable 'DNN/dense_2/kernel:0' shape=(512, 512) dtype=float32_ref>
<tf.Variable 'DNN/dense_3/kernel:0' shape=(512, 512) dtype=float32_ref>
<tf.Variable 'DNN/dense_4/kernel:0' shape=(512, 10) dtype=float32_ref>

('activations', <tf.Tensor 'Classifier/X:0' shape=(?, 784) dtype=float32>)
('activations', <tf.Tensor 'Classifier/DNN/dense/Relu:0' shape=(?, 512) dtype=float32>)
('activations', <tf.Tensor 'Classifier/DNN/dense_1/Relu:0' shape=(?, 512) dtype=float32>)
('activations', <tf.Tensor 'Classifier/DNN/dense_2/Relu:0' shape=(?, 512) dtype=float32>)
('activations', <tf.Tensor 'Classifier/DNN/dense_3/Relu:0' shape=(?, 512) dtype=float32>)
('activations', <tf.Tensor 'Classifier/DNN/dense_4/MatMul:0' shape=(?, 10) dtype=float32>)


In [14]:
def backward_pass_rel(top_rel, inputs, weights, outputs, epsilon=1e-4):
    return np.sum((inputs.T.dot(top_rel) * weights) / (outputs + epsilon),
                          axis=1,
                          keepdims=True).T

In [15]:
def backprop_taylor_rel(inputs, weights, top_rel, lowest=-1.5, highest=2.5):
    w_p = np.maximum(np.zeros_like(weights), weights)
    w_n = np.minimum(np.zeros_like(weights), weights)
    
    L = np.ones_like(inputs) * lowest
    H = np.ones_like(inputs) * highest
    
    z_o = inputs.dot(weights)
    z_p = L.dot(w_p)
    z_n = H.dot(w_n)
    
    z = z_o - z_p - z_n + 1e-10
    s = top_rel / z
    
    c_o = s.dot(weights.T)
    c_p = s.dot(w_p.T)
    c_n = s.dot(w_n.T)
    
    return inputs * c_o - L * c_p + H * c_n    

In [16]:
def cal_a_p(x, y,z,epsilon=1e-4):
    n_dx = x.shape[1]
    n_dy = y.shape[1]  #number of columns
    e_x = np.sum(x, axis=0) # axis=0, means add every row entry, eg. if it is a row array(horizontal vector), then the result is the same as the row array. So, here, np.shape(e_x)=np.shape(x)
    e_x = np.where(e_x > 0, e_x, 0)
    e_y = np.sum(y, axis=0)
    e_y2 = np.sum(y**2, axis=0)
    for i in range(n_dy):
        if i==0:
#             e_xy = np.sum(x * np.repeat(y[:,i:(i+1)], n_dx, axis=1), axis=0)
            e_xy = np.sum(x * y[:,[i]], axis=0)
            e_xy = np.where(e_xy>0, e_xy, 0)
            a = (e_xy - e_x * e_y[i]) / (z[:,i].T * e_xy - z[:,i].T * e_x * e_y[i] + epsilon)
        else:
#             e_xy = np.sum(x * np.repeat(y[:,i:(i+1)], n_dx, axis=1), axis=0)
            e_xy = np.sum(x * y[:,[i]], axis=0)
            e_xy = np.where(e_xy>0, e_xy, 0)
            a_n = (e_xy - e_x * e_y[i]) / (z[:,i].T * e_xy  - z[:,i].T * e_x * e_y[i] + epsilon)
            a = np.vstack((a, a_n))
    a = a.transpose()
    return a

In [17]:
def cal_a_n(x, y,z, epsilon=1e-4):
    n_dx = x.shape[1]
    n_dy = y.shape[1]
    e_x = np.sum(x, axis=0)
    e_x = np.where(e_x < 0, e_x, 0)
    e_y = np.sum(y, axis=0)
    e_y2 = np.sum(y**2, axis=0)
    for i in range(n_dy):
        if i==0:
            e_xy = np.sum(x * np.repeat(y[:,i:(i+1)], n_dx, axis=1), axis=0)  # repeat each colume n_dx times
            e_xy = np.where(e_xy<0, e_xy, 0)
            a = (e_xy - e_x * e_y[i]) / (z[:,i].T * e_xy - z[:,i].T * e_x * e_y[i] + epsilon)
        else:
            e_xy = np.sum(x * np.repeat(y[:,i:(i+1)], n_dx, axis=1), axis=0)
            e_xy = np.where(e_xy<0, e_xy, 0)
            a_n = (e_xy - e_x * e_y[i]) / (z[:,i].T * e_xy  - z[:,i].T * e_x * e_y[i] + epsilon)
            a = np.vstack((a, a_n))
    a = a.transpose()
    return a

In [18]:
weights_ = sess.run(weights)

In [19]:
activations_ = sess.run(activations, feed_dict={X: images})

In [20]:
a_dense5_p = cal_a_p(activations_[4], activations_[5], weights_[4])

In [21]:
a_dense4_p = cal_a_p(activations_[3], activations_[4], weights_[3])

In [22]:
a_dense3_p = cal_a_p(activations_[2], activations_[3], weights_[2])

In [ ]:
a_dense2_p = cal_a_p(activations_[1], activations_[2], weights_[1])

In [ ]:
a_dense1_p = cal_a_p(activations_[0], activations_[1], weights_[0])

In [ ]:
sample_imgs = []
for i in range(10):
#     sample_imgs.append(images[np.argmax(labels, axis=1) == i][3])
    sample_imgs.append(images[np.argmax(labels, axis=1) == i][i])

In [ ]:
activations_samples = sess.run(activations, feed_dict={X:sample_imgs})

In [ ]:
print(len(activations_samples[4]))
# print(activations_samples[3][0])

In [ ]:
# #PatternAttribute p
temp1_p = backprop_taylor_rel(activations_samples[4], weights_[4]*a_dense5_p, activations_samples[5])
temp1_p = backprop_taylor_rel(activations_samples[3], weights_[3]*a_dense4_p, temp1_p)
temp1_p = backprop_taylor_rel(activations_samples[2], weights_[2]*a_dense3_p, temp1_p)
temp1_p = backprop_taylor_rel(activations_samples[1], weights_[1]*a_dense2_p, temp1_p)
temp1_p = backprop_taylor_rel(activations_samples[0], weights_[0]*a_dense1_p, temp1_p)

In [ ]:
print(np.shape(temp1_p))

In [ ]:
# plt.imshow(np.reshape(temp1_n[2], [28,28]), cmap='gray')
# vmin, vmax = pixel_range(temp1_p[1])
plt.imshow(np.reshape(temp1_p[2], [28,28]),cmap='bwr')

In [ ]:
plt.figure(figsize=(15,15))
for i in range(5):
    plt.subplot(5, 2, 2 * i + 1)
#     plt.imshow(np.reshape(sample_imgs[2*i], [28,28]), cmap='gray')
    plt.imshow(np.reshape(temp1_p[2 * i], [28, 28]), cmap='bwr')
#     plt.imshow(np.reshape(temp1_p[2 * i], [28, 28]), cmap='hot_r')
#     plt.imshow(np.reshape(temp1_p[2], [28,28]), cmap='gray')
    plt.title('Digit: {}'.format(2 * i))
    plt.colorbar()
    
    plt.subplot(5, 2, 2 * i + 2)
#     plt.imshow(np.reshape(sample_imgs[2*i +1], [28,28]), cmap='gray')
#     plt.imshow(np.reshape(temp1_p[2 * i + 1], [28, 28]), cmap='bwr')
#     plt.imshow(np.reshape(temp1_p[2 * i + 1], [28, 28]), cmap='hot_r')
    plt.imshow(np.reshape(temp1_p[2 * i+1], [28,28]),cmap='bwr')
    plt.title('Digit: {}'.format(2 * i + 1))
    plt.colorbar()

plt.tight_layout()

In [ ]:
plt.figure(figsize=(15,15))
for i in range(5):
    plt.subplot(5, 2, 2 * i + 1)
    plt.imshow(np.reshape(sample_imgs[2 * i], [28, 28]), cmap='hot_r')
    plt.title('Digit: {}'.format(2 * i))
    plt.colorbar()
    
    plt.subplot(5, 2, 2 * i + 2)
    plt.imshow(np.reshape(sample_imgs[2 * i + 1], [28, 28]), cmap='hot_r')
    plt.title('Digit: {}'.format(2 * i + 1))
    plt.colorbar()

plt.tight_layout()

<font color=blue>__Calculating Relevance Scores $R(x_i)$__</font>

In [ ]:
# images = mnist.train.images
# labels = mnist.train.labels

# sample_imgs = []
# for i in range(10):
#     sample_imgs.append(images[np.argmax(labels, axis=1) == i][5])

In [ ]:
# print (np.shape(images))
# print(np.shape(labels))
# print(labels[44])
# print ('np.shape(images[np.argmax(labels, axis=1) == 0]) :',np.shape(images[np.argmax(labels, axis=1) == 0]))
# print (np.shape(images[np.argmax(labels, axis=1) == 0][3]))
# print('np.shape(sample_imgs): ',np.shape(sample_imgs))
# print(np.shape(sample_imgs)[1])
# print(np.shape(sample_imgs[1][None,:]))
# print(np.argmax(labels, axis=1) == 0)

In [ ]:

sess.close()

In [ ]:
# plt.figure(figsize=(15,15))
# for i in range(5):
#     plt.subplot(5,2,2*i+1)
#     plt.imshow(np.reshape(sample_imgs[2*i], [28,28]), cmap='gray')
    
#     vmin, vmax = pixel_range(hmaps[2 * i])
#     plt.imshow(np.reshape(hmaps[2*i], [28,28]), vmin=vmin, vmax=vmax, cmap='bwr', alpha=0.5)
#     plt.title('digit:{}'.format(2*i))
#     plt.colorbar()
    